In [ ]:
using Turing, Distributions
using PyPlot, PyCall
@pyimport matplotlib.cm as cm
@pyimport matplotlib.animation as animation
@pyimport numpy as np
@pyimport IPython.display as D

In [ ]:
# Load transition matrix T and observations obs
include(Pkg.dir("Turing")*"/notebooks/data/hmmdemo.data.jl");

In [ ]:
K = 5
N = 201
initial = fill(1.0 / K, K)
means = (collect(1.0:K)*2-K)*2

@model hmmdemo begin
    states = tzeros(Int,N)
    # T = TArray{Array{Float64,}}
    
    # Prior over T
    # for i=1:K, @assume T[i,:] ~ Dirichlet(ones(K)./K); end
    
    @assume states[1] ~ Categorical(initial)
    for i = 2:N
        @assume states[i] ~ Categorical(vec(T[states[i-1],:]))
        @observe obs[i] ~ Normal(means[states[i]], 4)
    end
    @predict states
end

In [ ]:
chain = sample(hmmdemo, PG(100,100));

In [ ]:
# Print the 1st sample trajectory
chain.value[1].value[:states]'

In [104]:
fig, ax = plt[:subplots]();
ax[:set_xlim](( 0, 210))
ax[:set_ylim]((-20, 20))
line, = ax[:plot]([], [], lw=2)

# initialization function: plot the background of each frame
function init()
    ax[:plot](1:N, obs, "rs")
    return (line,)
end

# animation function. This is called sequentially
function animate(i)
    n = mod(i,length(chain.value))
    line[:set_data](1:N, means[chain.value[n+1].value[:states]])
    return (line,)
end

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=200, blit=true)

plt[:close]()


In [105]:
D.HTML(anim[:to_html5_video]())

PyObject <IPython.core.display.HTML object>